In [156]:
import pandas as pd

<h1>1. Data Cleaning</h1>

In [158]:
df_historical_data=pd.read_csv('all_fifa_wc_resultsdata.csv')
df_fixture2022=pd.read_csv('2022fifa_wc_fixtures.csv')
df_missing_data=pd.read_csv('fifa_worldcup_missing_data.csv')

In [159]:
df_missing_data

,home,score,away,year
0,Mexico,0–0,Soviet Union,1970
1,Belgium,3–0,El Salvador,1970
2,Soviet Union,4–1,Belgium,1970
3,Mexico,4–0,El Salvador,1970
4,Soviet Union,2–0,El Salvador,1970
...,...,...,...,...
503,Spain,0–1,Switzerland,2010
504,Chile,1–0,Switzerland,2010
505,Spain,2–0,Honduras,2010
506,Chile,1–2,Spain,2010


<h3>1.1 Cleaning df_fixture2022<h3>

In [160]:
df_fixture2022['home'] = df_fixture2022['home'].str.strip()
df_fixture2022['away'] = df_fixture2022['away'].str.strip()

<h3>1.2 Cleaning the missing data and adding it to df_historical_data<h3>

In [161]:
#Null Data
#df_missing_data[df_missing_data['home'].isnull()]

#Drop Null Data
df_missing_data.dropna(inplace=True)

#Concatenating  missing data and historival data and cleaning them

df_historical_data = pd.concat([df_historical_data,df_missing_data],ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year',inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
413,Argentina,0–3,Croatia,2018
412,Croatia,2–0,Nigeria,2018
411,Argentina,1–1,Iceland,2018
410,Australia,0–2,Peru,2018


<h3>1.3 Cleaning df_historical_data<h3>

In [162]:
#Going through the dataset, found out that the match between sweden and austria had ended in a w/o, which could affect our prediction

delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') & df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index,inplace=True)

In [165]:
#Score columns that do not have digits and "-" 
#df_historical_data[df_historical_data['score'].str.contains('(a.e.t.)')]

#Replacing the score after removing (a.e.t.)At Extra Time
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]','',regex=True)

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_14128\3710528733.py:5: SyntaxWarning: invalid escape sequence '\d'
  df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]','',regex=True)


In [166]:
#Cleaning score and home/away columns
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

In [167]:
#Splitting score columns into home and away score columns
df_historical_data[['HomeGoals','AwayGoals']] = df_historical_data['score'].str.split('–',expand=True)

In [168]:
df_historical_data.drop('score',axis=1,inplace=True)

In [169]:
df_historical_data

,home,away,year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
413,Argentina,Croatia,2018,0,3
412,Croatia,Nigeria,2018,2,0
411,Argentina,Iceland,2018,1,1
410,Australia,Peru,2018,0,2


In [170]:
#Rename columns and change the datatypes
df_historical_data.rename(columns={'home':'HomeTeam','away':'AwayTeam','year':'Year'},inplace=True)

df_historical_data = df_historical_data.astype({'HomeGoals': int,'AwayGoals': int})

In [171]:
df_historical_data.dtypes

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals     int64
AwayGoals     int64
dtype: object

In [172]:
#Creating new column "TotalGoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']

In [173]:
df_historical_data['TotalGoals'].sum()

np.int64(2548)

In [174]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
413,Argentina,Croatia,2018,0,3,3
412,Croatia,Nigeria,2018,2,0,2
411,Argentina,Iceland,2018,1,1,2
410,Australia,Peru,2018,0,2,2


<h1>Exploring Clean Dataframes<h1>

In [175]:
df_historical_data.to_csv('clean_all_fifa_wc_resultsdata.csv', index= False)
df_fixture2022.to_csv('clean_2022fifa_wc_fixtures.csv',index=False)

<h1>Verifications<h1>

In [136]:
#Verifying number of matches per year
years = [1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))


1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64


In [137]:
#Verify Number of matches per competition

df_fixture2022['year'].value_counts()


year
2022    64
Name: count, dtype: int64